<div style="text-align: center;">
<strong>Machine Learning for Scientific Computing and Numerical Analysis - PC 4</strong>
</div>
<div style="text-align: center;">
<strong>Finite element approximation for time-dependant problems</strong>
</div>
<div style="text-align: center;">
<p>Loïc Gouarin, Samuel Kokh, Hadrien Montanelli</p>
</div>
<div style="text-align: center;">
<i>2024 - 2025</i>
</div>



In [1]:
from math import pi
import matplotlib.pyplot as plt
import numpy as np    
from numpy.linalg import inv, norm
from scipy.linalg import eigvals, expm
from scipy.sparse import diags, eye
from scipy.sparse.linalg import spsolve
import time
from helpers import heat_eq_exact
from helpers import plot_series

# 1 Introduction

### Exercise

{exercise}
Copy/paste the functions you defined in pc4_01.

In [2]:
# Shape function and basis functions:
phi = lambda x: np.where(np.abs(x) <= 1, 1 - np.abs(x), 0)
Phi = lambda j, x: phi((x - xx[j])/h)

# 2 Heat equation with homogeneous Dirichlet boundary conditions

In this section we are going to study a discretization of the one-dimensional heat equation
using the finite element method.

We note $\Omega=(-1,1)$ and $V=\{v\in H^1(-1,1),\,v(\pm1)=0\}$.

We consider the one-dimensional heat equation with homogeneous Dirichlet boundary conditions.

\begin{align}
u_t(x,t) &= u_{xx}(x,t) &&\text{for $x\in\Omega = (-1,1)$, $t\in(0,T]$,}
\\
u(-1,t) &= u(1,t) = 0, &&\text{for $t\in[0,T]$,}\\
u(x,0) &= u_0(x), &&\text{for $x\in\Omega = (-1,1)$.}\\
\end{align}

The weak formulation of the problem is to find $u(\cdot,t)\in V$ such that
$$
\int_{\Omega} u_t(x,t)v(x)dx = - \int_{\Omega} u_x(x,t)v_x(x)dx
$$
for all $v\in V$.

## 2.1 Spatial finite element approximation

We consider now the set of $m+2$ points $x_j = -1 + jh$, $h = 2/(m+1)$, for $j=0,\ldots,m+1$, which yields 
 a discretization grid of $[-1,1]$ and the associated basis functions $x\mapsto\phi_j(x)$, $j=1\ldots,m$.

We seek a finite element approximation of $u$
$$
u_h(x,t)
= \sum_{j=1}^m u_j(t)\phi_j(x)
$$

that verifies the approximate problem

$$
\int_{\Omega} \partial_t u_h(x,t)\phi_i(x)dx 
=
- \int_{\Omega} \partial_x u_h (x,t)\frac{d \phi_i}{dx}(x)dx
\quad \text{for all $i=1,\ldots,m$.}
 $$

By substituting the expression of $u_h$  we get the following system of ordinary differential equations

$$
\sum_{j=1}^m 
\frac{d u_j}{dt}(t)
\int_{\Omega} \phi_i(x)\phi_j(x)dx
=
-
 \sum_{j=1}^m
 u_j(t)
\int_{\Omega} \phi_i'(x)\phi_j'(x)dx
\quad \text{for all $i=1,\ldots,m$.}
$$ 

As previously, we set
$$
A_{ij} = \int_{\Omega} \phi_i'(x)\phi_j'(x)dx,
\quad
M_{ij} = \int_{\Omega} \phi_i(x)\phi_j(x)dx
,
$$
so that the system of ODEs reads
$$
M\frac{d U (t)}{dt}  = - A U (t),
$$
where 
$$
U(t) = (u_1(t),\ldots,u_m(t))^T,
\quad A = (A_{ij})_{1\leq i,j \leq m}
\quad M = (M_{ij})_{1\leq i,j\leq m}.
$$

We recall that if $T$ is the $m \times m$ tridiagonal matrix defined by $T_{i,i+1} = T_{i+1,i} = 1$ for $i=1,\ldots,m-1$ and $T_{i,j} = 0$ otherwise,

$$
T =
\begin{pmatrix}
0 & 1 & 0 & \cdots & 0\\
1 & 0 & 1 & \ddots & \vdots\\
0 & \ddots & \ddots & \ddots & 0\\
\vdots & \ddots & 1 & 0 & 1\\
0 & \cdots & 0 & 1 & 0\\
\end{pmatrix},
$$

then the matrices $A$ and $M$ can be expressed as 
$$
A =\frac{1}{h}
\left(
2 I - T
\right)
,
\qquad
M =
\frac{h}{3}
\left(
2I + \frac{1}{2}T
\right)
.
$$

### Exercise

{exercise}
Assemble the matrix A and M.

In [3]:
def assemble_matrices(m, h):
    T = diags([1] * (m - 1), offsets=1) + diags([1] * (m - 1), offsets=-1)
    A=1/h*(2*eye(m)-T)
    M=h/3*(2*eye(m)+1/2*T)
    return A, M

### Validation

In [4]:
m = 10
h = 2/(m + 1)
k = h**2/6 # comment

A, M = assemble_matrices(m, h)
print(A.todense()) # the first line of K is [11.  -5.5  0.   0.   0.   0.   0.   0.   0.   0. ]
print(M.todense()) # the first line of M is [0.12121212 0.03030303 0. 0. 0. 0. 0. 0. 0. 0. ]

[[11.  -5.5  0.   0.   0.   0.   0.   0.   0.   0. ]
 [-5.5 11.  -5.5  0.   0.   0.   0.   0.   0.   0. ]
 [ 0.  -5.5 11.  -5.5  0.   0.   0.   0.   0.   0. ]
 [ 0.   0.  -5.5 11.  -5.5  0.   0.   0.   0.   0. ]
 [ 0.   0.   0.  -5.5 11.  -5.5  0.   0.   0.   0. ]
 [ 0.   0.   0.   0.  -5.5 11.  -5.5  0.   0.   0. ]
 [ 0.   0.   0.   0.   0.  -5.5 11.  -5.5  0.   0. ]
 [ 0.   0.   0.   0.   0.   0.  -5.5 11.  -5.5  0. ]
 [ 0.   0.   0.   0.   0.   0.   0.  -5.5 11.  -5.5]
 [ 0.   0.   0.   0.   0.   0.   0.   0.  -5.5 11. ]]
[[0.12121212 0.03030303 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.03030303 0.12121212 0.03030303 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.03030303 0.12121212 0.03030303 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.03030303 0.12121212 0.03030303 0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.0303

## 2.2 Time integration with forward Euler (one-step Adams-Bashforth AB1)

In this section we propose to approximate the ODE system with forward Euler/one-step Adams-Bashforth.

Let $k>0$ be the time step, we use the time discretization
$$
k = T/(N+1)
,\quad
t_n = nk
,\quad
n=0,\ldots,N+1.
$$

We denote $U^n$ as an approximation of $U(t^n)$.

### Exercise (setup)

{exercise}
Show that the update from $U^{n}$ to $U^{n+1}$ takes the form
$$
U^{n+1} = U^n - L U^n,
$$
where $L$ is a matrix to be determined using $A$ and $M$.

On a: $$M\frac{U^{n+1}-U^n}{k}=-AU^{n}$$ D'où $$U^{n+1}=U^n-LU^n$$ avec $L=kM^{-1}AU^n$

### Exercise (implementation)

{exercise}
Complete the following function that computes the values of $(U^0,U^1,\ldots)$ with forward Euler (AB1) for all $n\in\mathbb{N}$ such that $0\leq t^n = n k \leq T$, given a time step $k>0$. The function returns a list of $U^n$ and $t^n$ for every $n$ that is a multiple of 100.

*Hint:* Handle the matrix inversion by solving linear systems using the `spsolve()` scipy routine.

In [5]:
def heat_AB1(k, h, T, m, u0):
    xx = np.linspace(-1, 1, m + 2) # computation grid
    all_U = []
    all_t = []
    U = u0(xx)
    U = U[1:-1]
    A, M = assemble_matrices(m, h)
    step = 0
    t = 0
    all_U.append(U)
    all_t.append(t)
    start = time.time() 
    while t < T:
        U = U-k*spsolve(M,eye(m))@A@U
        t = t + k
        step = step + 1
        if step % 100 == 0:
                all_U.append(U)
                all_t.append(t)
    end = time.time()
    print(f'Time  (tstep): {end-start:.5f}s')
    return all_t, all_U

### Validation

In [6]:
# Setup (problem):
u0 = lambda x: (1-x**2)*np.exp(-10*x**2) # initial condition

# Setup (discretization):
m = 100
h = 2/(m + 1)
k = h**2/6
T = 1000*k
xx = np.linspace(-1, 1, m + 2) # computation grid
xeval = np.linspace(-1, 1, 1000) # evaluation grid

# Solve the heat equation:
all_t, all_U = heat_AB1(k, h, T, m, u0)

# Evaluate the FEM approximation over the evaluation grid:
start = time.time()
def eval_fem(all_U: list, xeval: np.ndarray)-> list:
    n = all_U[0].shape[0]
    J=np.arange(1, n + 1)
    all_Ueval = []
    for U in all_U:
        I, S = np.meshgrid(J, xeval)
        Ueval = Phi(I, S) @ U
        all_Ueval.append(Ueval)    
    return all_Ueval
all_Ueval = eval_fem(all_U, xeval)
end = time.time()
print(f'Time   (eval): {end-start:.5f}s')

# Compute reference solution:
start = time.time() 
all_Uex = [heat_eq_exact(xeval, t, u0) for t in all_t]
end = time.time()
print(f'Time    (ref): {end-start:.5f}s')

# Plot the fem approximate solution with the reference solution:
plot_series(xeval, all_t, all_Ueval, "u_h", all_Uex, "u_ref", "Heat equation (forward Euler)")

C:\Users\ethan\AppData\Local\Temp\ipykernel_18648\255515714.py:14: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  U = U-k*spsolve(M,eye(m))@A@U
C:\Users\ethan\AppData\Local\Temp\ipykernel_18648\255515714.py:14: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  U = U-k*spsolve(M,eye(m))@A@U


Time  (tstep): 18.12956s
Time   (eval): 0.02429s
Time    (ref): 4.99661s


## 2.3 Time integration with backward Euler (one-step Adams-Moulton AM1)

In this section we propose to approximate the ODE system with backward Euler/one-step Adams-Moulton.

### Exercise (setup)

{exercise}
Show that the update from $U^{n}$ to $U^{n+1}$ takes the form
$$
L' U^{n+1} = M U^n,
$$
where $L'$ is a matrix to be determined using $A$ and $M$.

On a: $$M\frac{U^{n+1}-U^n}{k}=-AU^{n+1}$$ D'où $$L'U^{n+1}=MU^n$$ avec $L'=(M+kA)$

### Exercise (implementation)

{exercise}
Complete the following function that computes the values of $(U^0,U^1,\ldots)$ with backward Euler (AM1) for all $n\in\mathbb{N}$ such that $0\leq t^n = n k \leq T$, given a time step $k>0$. The function returns a list of $U^n$ and $t^n$ for every $n$ that is a multiple of 100.

*Hint:* Handle the matrix inversion by solving linear systems using the `spsolve()` scipy routine.

In [7]:
def heat_AM1(k, h, T, m, u0):
    xx = np.linspace(-1, 1, m + 2) # computation grid
    all_U = []
    all_t = []
    U = u0(xx)
    U = U[1:-1]
    A, M = assemble_matrices(m, h)
    step = 0
    t = 0
    all_U.append(U)
    all_t.append(t)
    start = time.time() 
    while t < T:
        L_prime=M+k*A
        U = spsolve(L_prime,M@U)
        t = t + k
        step = step + 1
        if step % 100 == 0:
                all_U.append(U)
                all_t.append(t)
    end = time.time()
    print(f'Time  (tstep): {end-start:.5f}s')
    return all_t, all_U

### Validation

In [8]:
# Setup (problem):
u0 = lambda x: (1-x**2)*np.exp(-10*x**2) # initial condition

# Setup (discretization):
m = 100
h = 2/(m + 1)
k = h**2/6
T = 1000*k
xx = np.linspace(-1, 1, m + 2) # computation grid
xeval = np.linspace(-1, 1, 1000) # evaluation grid

# Solve the heat equation:
all_t, all_U = heat_AM1(k, h, T, m, u0)

# Evaluate the FEM approximation over the evaluation grid:
start = time.time()
all_Ueval = eval_fem(all_U, xeval)
end = time.time()
print(f'Time   (eval): {end-start:.5f}s')

# Compute reference solution:
start = time.time() 
all_Uex = [heat_eq_exact(xeval, t, u0) for t in all_t]
end = time.time()
print(f'Time    (ref): {end-start:.5f}s')

# Plot the fem approximate solution with the reference solution:
plot_series(xeval, all_t, all_Ueval, "u_h", all_Uex, "u_ref", "Heat equation (backward Euler)")

Time  (tstep): 0.40473s
Time   (eval): 0.02264s
Time    (ref): 4.54935s


## 2.4 Spectrum analysis

Let us examine the spectrum of the matrices T, A and M previously computed and compare them with the analytical formulas:

$$
\begin{align*}
\operatorname{sp}(T) 
&=
\left\{
-2 + 4 \cos^2
\left(
\frac{j\pi}{2(m+1)}
\right)
,\
j=1,\ldots,m
\right\},
\\
\operatorname{sp}(A) 
&=
\left\{
\frac{4}{h}
\sin^2
\left(
\frac{j\pi}{2(m+1)}
\right)
,\
j=1,\ldots,m
\right\},
\\
\operatorname{sp}(M) 
&=
\left\{
\frac{h}{3}
\left(
1+
2\cos^2
\left(
\frac{j\pi}{2(m+1)}
\right)
\right)
,\
j=1,\ldots,m
\right\}.
\end{align*}
$$

### Exercise

{exercise}
Compute the spectrum of T, A, M as a sorted numpy array and compare the result with the analytical formulas.

*Hints:*
* transform the sparse matrice into dense matrices and use the numpy `eigvals()` function;
* sort the element of a numpy arrays using the numpy `sort()` function.

In [9]:
m = 100
h = 2/(m + 1)
T = diags([1] * (m - 1), offsets=1) + diags([1] * (m - 1), offsets=-1)
A,M=assemble_matrices(m, h)
T=T.todense()
A=A.todense()
M=M.todense()

# eigenvalues of T
egval_T = np.sort(eigvals(T))
egval_T_ref = np.sort([-2+4*np.cos(j*np.pi/(2*(m+1)))**2 for j in range(1,m+1)])
print(np.linalg.norm(egval_T - egval_T_ref))

# eigenvalues of A
egval_A = np.sort(eigvals(A))
egval_A_ref = np.sort([4/h*np.sin(j*np.pi/(2*(m+1)))**2 for j in range(1,m+1)])
print(np.linalg.norm(egval_A - egval_A_ref))

# eigenvalues of M
egval_M = np.sort(eigvals(M))
egval_M_ref = np.sort([h/3*(1+2*np.cos(j*np.pi/(2*(m+1)))**2) for j in range(1,m+1)])
print(np.linalg.norm(egval_M - egval_M_ref))

3.8862612142641275e-14
2.783413858016559e-12
3.807115373124776e-16


## 2.5 Stability analysis

We recall that the spectral radius $\sigma(R)$ of a square matrix $R$  is $\sigma(R) = \max_{\lambda\in\operatorname{sp}(R)}|\lambda|$. It is easy to show that

$$
\sigma(A) \leq 4/h, \quad \sigma(M^{-1}) \leq 3/h,
$$

and that

$$
\sigma(M^{-1}A) \leq \sigma(M^{-1})\sigma(A) \leq 12/h^2.
$$

The resulting stability restriction for the time-step $k$ for forward Euler reads $\vert k\lambda(-M^{-1}A) + 1\vert\leq1$, that is,

$$
-1 \leq 1 - 12k/h^2 \leq 1,
$$

which yields $k\leq h^2/6$. 

For backward Euler, $\vert k\lambda(-M^{-1}A) - 1\vert \geq1$ does not give any restriction.

### Exercise
{exercise}
Recompute the solution of sections 2.2 and 2.3 with $m=100$ and time-step $k=2h^2/6$ to illustrate numerical instability.

In [10]:
# Setup (problem):
u0 = lambda x: (1-x**2)*np.exp(-10*x**2) # initial condition

# Setup (discretization):
m = 100
h = 2/(m + 1)
k = 2*h**2/6
T = 1000*k
xx = np.linspace(-1, 1, m + 2) # computation grid
xeval = np.linspace(-1, 1, 1000) # evaluation grid

# Solve the heat equation:
all_t, all_U = heat_AB1(k, h, T, m, u0)

# Evaluate the FEM approximation over the evaluation grid:
start = time.time()
def eval_fem(all_U: list, xeval: np.ndarray)-> list:
    n = all_U[0].shape[0]
    J=np.arange(1, n + 1)
    all_Ueval = []
    for U in all_U:
        I, S = np.meshgrid(J, xeval)
        Ueval = Phi(I, S) @ U
        all_Ueval.append(Ueval)    
    return all_Ueval
all_Ueval = eval_fem(all_U, xeval)
end = time.time()
print(f'Time   (eval): {end-start:.5f}s')

# Compute reference solution:
start = time.time() 
all_Uex = [heat_eq_exact(xeval, t, u0) for t in all_t]
end = time.time()
print(f'Time    (ref): {end-start:.5f}s')

# Plot the fem approximate solution with the reference solution:
plot_series(xeval, all_t, all_Ueval, "u_h", all_Uex, "u_ref", "Heat equation (forward Euler)")

C:\Users\ethan\AppData\Local\Temp\ipykernel_18648\255515714.py:14: SparseEfficiencyWarning:

spsolve requires A be CSC or CSR matrix format

C:\Users\ethan\AppData\Local\Temp\ipykernel_18648\255515714.py:14: SparseEfficiencyWarning:

spsolve is more efficient when sparse b is in the CSC matrix format



Time  (tstep): 16.82039s
Time   (eval): 0.02390s
Time    (ref): 4.48960s


In [11]:
# Setup (problem):
u0 = lambda x: (1-x**2)*np.exp(-10*x**2) # initial condition

# Setup (discretization):
m = 100
h = 2/(m + 1)
k = 2*h**2/6
T = 1000*k
xx = np.linspace(-1, 1, m + 2) # computation grid
xeval = np.linspace(-1, 1, 1000) # evaluation grid

# Solve the heat equation:
all_t, all_U = heat_AM1(k, h, T, m, u0)

# Evaluate the FEM approximation over the evaluation grid:
start = time.time()
all_Ueval = eval_fem(all_U, xeval)
end = time.time()
print(f'Time   (eval): {end-start:.5f}s')

# Compute reference solution:
start = time.time() 
all_Uex = [heat_eq_exact(xeval, t, u0) for t in all_t]
end = time.time()
print(f'Time    (ref): {end-start:.5f}s')

# Plot the fem approximate solution with the reference solution:
plot_series(xeval, all_t, all_Ueval, "u_h", all_Uex, "u_ref", "Heat equation (backward Euler)")

Time  (tstep): 0.48658s
Time   (eval): 0.03124s
Time    (ref): 5.28686s


# 3 Allen-Cahn equation with homogeneous Dirichlet boundary conditions

Let $\varepsilon>0$. We consider the one-dimensional Allen-Cahn equation with homogeneous Dirichlet boundary conditions;

\begin{align}
u_t(x,t) &= \varepsilon u_{xx}(x,t) + u(x,t) - u(x,t)^3 &&\text{for $x\in\Omega = (-1,1)$, $t\in(0,T]$,}
\\
u(-1,t) &= u(1,t) = 0, &&\text{for $t\in[0,T]$,}\\
u(x,0) &= u_0(x), &&\text{for $x\in\Omega = (-1,1)$.}\\
\end{align}

The weak formulation of the problem is to find $u(\cdot,t)\in V$ such that

$$
\int_{\Omega} u_t(x,t)v(x)dx 
=
- \int_{\Omega} u_x(x,t)v_x(x)dx
+ \int_{\Omega} u(x,t)v(x)dx
+ \int_{\Omega} u^3(x,t)v(x)dx
$$
for all $v\in V$.

## 3.1 Spatial finite element approximation

We consider now the set of $n+2$ points $x_j = -1 + jh$, $h = 2/(m+1)$, for $j=0,\ldots,m+1$, which yields a discretization grid of $[-1,1]$ and associated basis functions $x\mapsto\phi_j(x)$, $j=1\ldots,m$.

We seek a finite element approximation of $u$ 

$$
u_h(x,t)
= \sum_{j=1}^m u_j(t)\phi_j(x)
$$

that verifies the approximate problem

$$
\int_{\Omega} \partial_t u_h(x,t)\phi_i(x)dx 
=
-
\varepsilon
\int_{\Omega} \partial_x u_h (x,t)\frac{d \phi_i}{dx}(x)dx
+
\int_{\Omega} u_h (x,t)\phi_i(x)dx
-
R_i
\quad \text{for all $i=1,\ldots,m$,}
$$

where

$$
\text{$R_i$ is an approximation of}
\quad
\int_{\Omega} u_h (x,t)^3 \phi_i(x)dx
\quad \text{for all $i=1,\ldots,m$.}
$$

We note again
$$
A_{ij} = \int_{\Omega} \phi_i'(x)\phi_j'(x)dx,
\quad
M_{ij} = \int_{\Omega} \phi_i(x)\phi_j(x)dx
,
\quad
U(t) = (u_1(t),\ldots,u_m(t))^T,
\quad
A= (A_{ij})_{1\leq i,j \leq m}
\quad
M = (M_{ij})_{1\leq i,j\leq m}.
$$

### Exercise

{exercise}
We choose to set

$$
R_i=
\sum_{j=1}^m
\int_{\Omega} u_j (t)^3 \phi_j(x)\phi_i(x)dx
\quad \text{for all $i=1,\ldots,m$.}
$$

Using the Hadamard product $U(t)\odot U(t) \odot U(t) = (u_1(t)^3,\cdots,u_m(t)^3)^T$, $A$ and $M$, write the system of ordinary differential equations verified by $t\mapsto U(t)$.

**Réponse à l'exercice 9**

On va introduire l'expression $$ u_h(x,t) = \sum_{j=1}^m u_j(t)\,\phi_j(x), $$ dans l'équation (25)

On a d'abord: 

   $$
   \begin{aligned}
   \int_{\Omega} \partial_t u_h(x,t) \phi_i(x)\,dx 
   &= \sum_{j=1}^m u_j'(t) \int_{\Omega}\phi_j(x)\phi_i(x)\,dx 
   &= \sum_{j=1}^m M_{ij}\, u_j'(t).
   \end{aligned}
   $$

   On a donx $M\, U'(t)$

Ensuite,

   $$
   \begin{aligned}
   -\varepsilon\int_{\Omega} u_h'(x,t) \phi_i'(x)\,dx 
   &= -\varepsilon\sum_{j=1}^m u_j(t) \int_{\Omega}\phi_j'(x)\phi_i'(x)\,dx 
   &= -\varepsilon\sum_{j=1}^m A_{ij}\, u_j(t).
   \end{aligned}
   $$

  On a donc  $ -\varepsilon\, A\, U(t).$

De plus,

   $$
   \begin{aligned}
   \int_{\Omega} u_h(x,t)\phi_i(x)\,dx 
   &= \sum_{j=1}^m u_j(t) \int_{\Omega}\phi_j(x)\phi_i(x)\,dx 
   &= \sum_{j=1}^m M_{ij}\, u_j(t).
   \end{aligned}
$$

On a donc $ M\, U(t). $

Enfin,

   $$
   R_i = \sum_{j=1}^m u_j(t)^3 \int_{\Omega}\phi_j(x)\phi_i(x)\,dx.
   $$

Ainsi,

   $$
   R = M\bigl(U(t)\odot U(t)\odot U(t)\bigr).
   $$

Au final, on a l'expression suivante:

$$
\boxed{M\, U'(t) = -\varepsilon\, A\, U(t) + M\, U(t) - M\bigl(U(t)\odot U(t)\odot U(t)\bigr).}
$$

## 3.2 Time integration with forward Euler

Let $k>0$ be the time step, we use the time discretization
$$
k = T/(N+1)
,\quad
t_n = nk
,\quad
n=0,\ldots,N+1.
$$

We denote $U^n$ as an approximation of $U(t^n)$.

### Exercise (setup)

{exercise}
Perform a time discretization of the ODE system using the forward Euler method. Show that it takes the form
$$
U^{n+1} = U^{n} + k V^{n}
,
$$
where $V^{n}$ is to be determined using $M$, $A$, $U^{n}$ and $U^{n}\odot U^{n} \odot U^{n}$.

On part de

$$
M\, U'(t) = -\varepsilon\, A\, U(t) + M\, U(t) - M\bigl(U(t)\odot U(t)\odot U(t)\bigr),
$$

et on utlise le fait que: 
$$
U'(t^n) \approx \frac{U^{n+1} - U^n}{k}.
$$

En substituant dans l'ODE, on obtient: 
$$
U^{n+1} = U^n + k\, V^n
$$
avec 
$$
V^n = -\varepsilon\, M^{-1}\, A\, U^n + U^n - U^n\odot U^n\odot U^n
$$



### Exercise (implementation)

{exercise}
Complete the following function that computes the values of $(U^0,U^1,\ldots)$ with forward Euler (AB1) for all $n\in\mathbb{N}$ such that $0\leq t^n = n k \leq T$, given a time step $k>0$. The function returns a list of $U^n$ and $t^n$ for every $n$ that is a multiple of 100.

*Hint:* Handle the matrix inversion by solving linear systems using the `spsolve()` scipy routine.

In [12]:
def Allen_Cahn_AB1(k, h, T, m, u0, eps):
    xx = np.linspace(-1, 1, m + 2) # computation grid
    all_U = []
    all_t = []
    U = u0(xx)
    U = U[1:-1]
    A, M = assemble_matrices(m, h)
    step = 0
    t = 0
    all_U.append(U)
    all_t.append(t)
    start = time.time() 
    while t < T:
        RHS = -eps * (A @ U) + M @ (U - U**3)
        V = spsolve(M, RHS)
        U = U+k*V # replace me: update the value of U
        t = t + k
        step = step + 1
        if step % 100 == 0:
                all_U.append(U)
                all_t.append(t)
    end = time.time()
    print(f'Time  (tstep): {end-start:.5f}s')
    return all_t, all_U

### Validation

 Solve the Allen-Cahn equation with the Adam-Bashforth order 1 scheme.

In [13]:
# Setup (problem):
eps = 0.01
u0 = lambda x: np.exp(-100*(x+0.5)**2) - np.exp(-100*(x-0.5)**2) # initial condition

# Setup (discretization):
m = 100
h = 2/(m + 1)
xx = np.linspace(-1, 1, m + 2) # computation grid
xeval = np.linspace(-1, 1, 1000) # evaluation grid
k = 1/eps*h**2/6
T = 1000*k
N = int(T/k - 1)

# Solve the Allen Cahn equation:
all_t, all_U = Allen_Cahn_AB1(k, h, T, m, u0, eps)

# Evaluate the FEM approximation over the evaluation grid:
all_Ueval = eval_fem(all_U, xeval)

# Import reference solution at final time:
uex = np.loadtxt('uex-ac.txt')

# Plot the fem approximate solution:
plot_series(xeval, all_t, all_Ueval, "u_h", title="Allen-Cahn (forward Euler)")

# Compare the approximate solution with the reference solution final time:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=xeval, y=all_Ueval[-1], mode='markers', name='uh'))
fig.add_trace(go.Scatter(x=xeval, y=uex, mode='lines', name='uex'))
fig.update_layout(title='Allen-Cahn (forward Euler)', xaxis_title='x', yaxis_title='u')
fig.show()

# Evaluation of the L-inf relative error at the final time with respect to the reference solution:
err = np.linalg.norm(all_Ueval[-1] - uex,np.inf) / np.linalg.norm(uex,np.inf)
print(f'Error at final time: {err:.2e}')

C:\Users\ethan\AppData\Local\Temp\ipykernel_18648\3273765821.py:15: SparseEfficiencyWarning:

spsolve requires A be CSC or CSR matrix format



Time  (tstep): 0.34828s


Error at final time: 2.86e-03
